In [1]:
# install these two libraries
!pip install snowflake-ml-python
!pip install snowflake-snowpark-python

In [5]:
!pip install snowflake-connector-python

In [8]:
import snowflake.connector

In [9]:
from snowflake.snowpark import Session
#from credential import params

In [10]:
conn = snowflake.connector.connect(
    user='SidelnikovStanislav',
    password='Gg8dE8FgjAR6qRm',
    account='XKGMVAT-EW81401'
)

In [12]:
%pip install --upgrade ipywidgets
!jupyter nbextension enable --py widgetsnbextension

   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.2 MB 4.2 MB/s eta 0:00:01
   ----------------------- ---------------- 1.3/2.2 MB 3.2 MB/s eta 0:00:01
   -------------------------------------- - 2.1/2.2 MB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 3.3 MB/s  0:00:00

   ------------- -------------------------- 1/3 [jupyterlab_widgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   ---------------------------------------- 3/3 [ipywidgets]

Note: you may need to restart the kernel to use updated packages.


usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook run server troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


In [13]:
from snowflake.ml.modeling.xgboost import XGBClassifier

In [14]:
from snowflake.snowpark.functions import col

In [15]:
from snowflake.ml.modeling import preprocessing

In [16]:
# Here’s the neighborhood visiting pattern the truck follows:
# In January, the truck goes to N1 on the 1st, 8th, 15th, 22nd, and 29th, and N2 the other days.

# From February through November, it goes to:
# N1 on the 1st
# N2 on the 2nd
# N3 on the 3rd
# N4 on the 4th
# N5 on the 5th
# N6 on the 6th
# N7 on the 7th
# N1 on the 8th
# N2 on the 9th
# etc.

# Every December, it only goes to N8.

month_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

In [ ]:

pre = {}

In [28]:
for i,month_length in enumerate(month_days):
    month = i + 1
    print(month, month_length)

1 31
2 28
3 31
4 30
5 31
6 30
7 31
8 31
9 30
10 31
11 30
12 31


In [ ]:
print(month_length)

31


In [26]:
for i,month_length in enumerate(month_days):
    month = i + 1

    for day in range(1,month_length+1):
        
        # In January, it goes to neighborhood 1 on Mondays, and neighborhood 2 the other days.
        if ((month) == 1):
            if (day) % 7 == 1:
                pre[(month,day)] = 1
            else:
                pre[(month,day)] = 2
                
        # From February through November, it goes to neighborhood 1 on the 1st, 2 on the 2nd, 3 on the 3rd,
        # 4 on the 4th, 5 on the 5th, 6 on the 6th, and 7 on the 7th, 1 on the 8th, 2 on the 9th, etc.
        elif ((month) <= 11):
            pre[(month,day)] = ((day-1) % 7) + 1

        # Every December, it only goes to neighborhood 8.
        elif ((month) == 12):
            pre[(month,day)] = 8


NameError: name 'month_days' is not defined

In [25]:
pre

NameError: name 'pre' is not defined

In [16]:
import pandas as pd

In [17]:
# Example Pandas DataFrame
df = pd.DataFrame({
    "id": [1, 2, 3],
    "name": ["apple", "banana", "cherry"]
})


In [20]:
print(pre)

NameError: name 'pre' is not defined

In [ ]:


# Connect to Snowflake
conn = connect(params)
cs = conn.cursor()

In [ ]:

# Write Pandas DataFrame to Snowflake table
success, nchunks, nrows, _ = write_pandas(conn, df, "DF_CLEAN")
print(f"Inserted {nrows} rows into DF_CLEAN")

In [2]:
from credentials import params

In [4]:
from snowflake.snowpark import Session
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.snowpark.functions import col
from snowflake.ml.modeling import preprocessing

In [5]:
session = Session.builder.configs(params).create()

In [8]:

# create a dataframe (though note that this doesn’t pull data into your local machine)
snowpark_df = session.table("TASTY_BYTES.RAW_POS.truck")


In [9]:

# show the first forty rows of the dataframe
snowpark_df.show(n=40)


------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"TRUCK_ID"  |"MENU_TYPE_ID"  |"PRIMARY_CITY"  |"REGION"    |"ISO_REGION"  |"COUNTRY"      |"ISO_COUNTRY_CODE"  |"FRANCHISE_FLAG"  |"YEAR"  |"MAKE"        |"MODEL"            |"EV_FLAG"  |"FRANCHISE_ID"  |"TRUCK_OPENING_DATE"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1           |1               |San Mateo       |California  |CA            |United States  |US                  |0                 |2009    |Ford_         |Step Van           |0          |1               |2019-01-01            |
|2           |2               |San Mateo       |California  |CA            |United S

In [ ]:

# count the rows in the dataframe
snowpark_df.count()

# describe the dataframe
snowpark_df.describe().show()

# groupby neighborhood, and show the counts
snowpark_df.group_by("Neighborhood").count().show()

# one way to scale your target (neighborhood) so you can use it in the XGBClassifier model
test = snowpark_df.withColumn('NEIGHBORHOOD2', snowpark_df.neighborhood - 1).drop("Neighborhood")

test.show()

# now use scikit-learn's LabelEncoder -- a more general solution -- through Snowpark ML 
le = LabelEncoder(input_cols=['NEIGHBORHOOD'], output_cols= ['NEIGHBORHOOD2'], drop_input_cols=True)

# apply the LabelEncoder
fitted = le.fit(snowpark_df.select("NEIGHBORHOOD"))

snowpark_df_prepared = fitted.transform(snowpark_df)

snowpark_df_prepared.show()

# split the data into a training set and a test set
train_snowpark_df, test_snowpark_df = snowpark_df_prepared.randomSplit([0.9, 0.1])

# save training data
train_snowpark_df.write.mode("overwrite").save_as_table("df_clean_train")

# save test data
test_snowpark_df.write.mode("overwrite").save_as_table("df_clean_test")

# create and train the XGBClassifier model
FEATURE_COLS = ["MONTH", "DAY"]
LABEL_COLS = ["NEIGHBORHOOD2"]

# Train an XGBoost model on snowflake.
xgboost_model = XGBClassifier(
    input_cols=FEATURE_COLS,
    label_cols=LABEL_COLS
)

xgboost_model.fit(train_snowpark_df)

# check the accuracy using scikit-learn's score functionality through Snowpark ML
accuracy = xgboost_model.score(test_snowpark_df)

print("Accuracy: %.2f%%" % (accuracy * 100.0))